# callbacks.general_sched

Creates a scheduler that lets you train a model with following different `TrainingPhase`.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.general_sched import * 
from fastai import *
from fastai.docs import *

In [ ]:
show_doc(TrainingPhase, doc_string=False)

## <a id=TrainingPhase></a>`class` `TrainingPhase`
> `TrainingPhase`(`length`:`int`, `lrs`:`Floats`, `moms`:`Floats`, `lr_anneal`:`AnnealFunc`=`None`, `mom_anneal`:`AnnealFunc`=`None`)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/general_sched.py#L8">[source]</a>

Create a phase for training a model during `length` iterations, following a schedule given by `lrs` and `lr_anneal`, `moms` and `mom_anneal`. More specifically, the phase will make the learning rate (or momentum) vary from the first value of `lrs` (or `moms`) to the second, following `lr_anneal` (or `mom_anneal`). If an annealing function is speficied but `lrs` or `moms` is a float, it will decay to 0. If no annealing function is specified, the default is a linear annealing if `lrs` (or `moms`) is a tuple, a constant parameter if it's a float. 

In [ ]:
show_doc(GeneralScheduler)

## <a id=GeneralScheduler></a>`class` `GeneralScheduler`
> `GeneralScheduler`(`learn`:[`Learner`](/basic_train.html#Learner), `phases`:`Collection`\[[`TrainingPhase`](/callbacks.general_sched.html#TrainingPhase)\]) :: [`Callback`](/callback.html#Callback)


Schedule multiple [`TrainingPhase`](/callbacks.general_sched.html#TrainingPhase) for a [`Learner`](/basic_train.html#Learner). <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/general_sched.py#L21">[source]</a>

In [ ]:
show_doc(GeneralScheduler.on_batch_end, doc_string=False)

#### <a id=on_batch_end></a>`on_batch_end`
> `on_batch_end`(`kwargs`:`Any`)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/general_sched.py#L34">[source]</a>

Takes a step in the current phase and prepare the hyperparameters for the next batch.

In [ ]:
show_doc(GeneralScheduler.on_train_begin, doc_string=False)

#### <a id=on_train_begin></a>`on_train_begin`
> `on_train_begin`(`n_epochs`:`int`, `kwargs`:`Any`)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/callbacks/general_sched.py#L26">[source]</a>

Initiates the hyperparameters to the start values of the first phase. 

Let's make an example by using this to code [SGD with warm restarts](https://arxiv.org/abs/1608.03983).

In [ ]:
def fit_sgd_warm(learn, n_cycles, lr, mom, cycle_len, cycle_mult):
    phases = [TrainingPhase((cycle_len)**(i+1), lr, mom, lr_anneal=annealing_cos) for i in range(n_cycles)]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    total_epochs = int((cycle_len - (cycle_len)**(n_cyles+1))/(1-cycle_len)) 
    learn.fit(total_epochs)

In [ ]:
data = get_mnist()
learn = ConvLearner(data, tvm.resnet18)
fit_sgd_warm(learn, 3, 1e-3, 0.9, 1, 3)

FileNotFoundError: [Errno 2] No such file or directory: '../data/mnist_sample.tgz'

## Undocumented Methods - Methods moved below this line will intentionally be hidden